In [ ]:
# importante agregar el uuid

In [7]:
import pandas as pd
import mlflow.sklearn

from datetime import datetime
from kedro.io.core import DatasetError
from mlflow.exceptions import MlflowException
from mlflow import MlflowClient

from src.preprocesamiento.preprocesamiento import crear_preprocesador, limpieza
from src.config.config import load_catalog

In [8]:
catalog, params = load_catalog(env="base")

client = MlflowClient()
date = datetime.now()

nombre_usecase = params['nombre_usecase']
alias = params['aliases_champion']
#cargar datos nuevos.
try:
    df_to_predict = catalog.load('to_predict_intermediate')

except DatasetError as e:
    print("Error la data no se encuentra")
    print(e)
    exit(1)

# Cargar modulo de preprocesamiento
df_limpio = limpieza(df_to_predict)

# cargar modelo
model = mlflow.sklearn.load_model(f"models:/{nombre_usecase}@champion")
model_version = client.get_model_version_by_alias(name=nombre_usecase,alias=alias)
# features
num_features = params['num_features']
cat_features = params['cat_features']

# Df's
X = df_limpio[num_features + cat_features]

predicciones = model.predict(X)

X['prediction'] =  predicciones
X['version'] = model_version.version
X['timestamp'] = date
X['label'] = pd.NA


catalog.save(ds_name='predicciones_modelo',data=X)
X.head()

D:\DATOS\Users\darodriguezo\AppData\Local\Temp\1\ipykernel_13096\1187373761.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['prediction'] =  predicciones
D:\DATOS\Users\darodriguezo\AppData\Local\Temp\1\ipykernel_13096\1187373761.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['version'] = model_version.version
D:\DATOS\Users\darodriguezo\AppData\Local\Temp\1\ipykernel_13096\1187373761.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,Age,Fare,SibSp,Parch,Sex,Embarked,Pclass,prediction,version,timestamp,label
0,30.00,24.1500,1,1,female,s,3,0,1,2026-02-18 14:18:17.265495,<NA>
1,34.00,13.0000,0,0,male,s,2,0,1,2026-02-18 14:18:17.265495,<NA>
2,31.00,26.2500,1,1,female,s,2,1,1,2026-02-18 14:18:17.265495,<NA>
3,11.00,120.0000,1,2,male,s,1,1,1,2026-02-18 14:18:17.265495,<NA>
4,0.42,8.5167,0,1,male,c,3,1,1,2026-02-18 14:18:17.265495,<NA>
